H3 Count of Incorrect Answers 

In [1]:
import pandas as pd
import pickle
from scipy import stats
import pandas as pd

In [2]:
#import
df = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )


In [3]:
def falschLabel(row):
    if row['Erfolg'] == 1:
        return 0
    return 1

xmlsaetze['falsch']= xmlsaetze.apply (lambda row: falschLabel(row), axis=1)
xmlsaetze['cnt_fehler'] = xmlsaetze.falsch + xmlsaetze.MehrfachFalsch

grouped = xmlsaetze.groupby(['UserID','UebungsID']).agg({'ID':'count', 'cnt_fehler':'sum'})
grouped = grouped.reset_index() 
grouped = grouped.rename(columns={'ID':'Count_XMLsaetze'})

In [4]:
intvgroup = df[['UebungsID','interventiongroup','Geschlecht','Klassenstufe']]
grouped = pd.merge(grouped, intvgroup, on='UebungsID',how='left')
grouped_user = grouped.groupby(['UserID','interventiongroup','Geschlecht','Klassenstufe']).agg({'cnt_fehler':'sum', 'Count_XMLsaetze':'count'})
grouped_user = grouped_user.reset_index()
#grouped_user

In [5]:
grouped_user['prop_fehler'] = grouped_user.cnt_fehler / grouped_user.Count_XMLsaetze

In [6]:
final = grouped_user[['interventiongroup','prop_fehler','Geschlecht','Klassenstufe','UserID']]

In [7]:
final.to_pickle('../00_data/final/incorrect_answers.pkl')